# Pandas Part1

## PySpark vs Pandas vs Numpy
Pandas DataFrame是一个二维数组，底层使用Numpy和array存储，Numpy使用C语言编写，运行速度很快

Spark和Pandas都可以集成SQL能力，但他们支持的SQL规范不一致，为了保持统一，需要做数据转换

PySpark DataFram转Pandas DataFrame使用toPandas()方法

Pandas DataFrame转PySpark DataFrame，使用sqlContext.createDataFrame(pdf)

### 导入库

In [ ]:
import pandas as pd
import numpy as np

## 创建数据
pandas支持创建单列数据 Series和多列数据 DataFrame

### Series 数据
Series数据是一维数据，Index可以是一个，也可以有多个

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

使用boolean array过滤Series数据，应用数学函数

In [ ]:
print(s[s<5])
print(s*2)
print(np.exp(s))

0    1.0
1    3.0
dtype: float64
0     2.0
1     6.0
2    10.0
3     NaN
4    12.0
5    16.0
dtype: float64
0       2.718282
1      20.085537
2     148.413159
3            NaN
4     403.428793
5    2980.957987
dtype: float64


通过`values`和`index`属性，可以获取数组的值和索引对象

In [ ]:
print(3 in s)
print(s.values)
print(s.index)

True
[ 1.  3.  5. nan  6.  8.]
RangeIndex(start=0, stop=6, step=1)


可以重新指定index的指

In [ ]:
s.index = ['a','b','c','d','e','f']
s

a    1.0
b    3.0
c    5.0
d    NaN
e    6.0
f    8.0
dtype: float64

可以把Series当做一个固定长度的、排序的字典, 可以传入一个dict来创建Series

In [ ]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

也可以指定自己的索引来获取指定的数据

In [ ]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states) # 只会获取索引在sdata中发现的数据
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

查看空数据,可以使用pd的isnull方法，也可以直接调用seris的isnull方法

In [ ]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [ ]:
obj4.notnull()

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [ ]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series对象和它的index对象都有一个name属性

In [ ]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

创建Series的同时，可以指定index

In [ ]:
ingredients = pd.Series(['4 cups', '1 cup', '2 large', '1 can'], index=['Flour', 'Milk', 'Eggs', 'Spam'], name='Dinner')
ingredients

Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object

使用index中的labels来访问一个值，或者一组值

In [ ]:
print(ingredients['Flour'])
print(ingredients[['Flour','Milk']])

4 cups
Flour    4 cups
Milk      1 cup
Name: Dinner, dtype: object


Series 和 DataFrame数据转换

In [ ]:
import pandas as pd

data_list = [(('一级科员', '0-20'), 0), (('一级科员', '21-30'), 14), (('一级科员', '31-40'), 10), 
             (('一级科员', '41-50'), 5), (('一级科员', '51-60'), 6), (('一级科员', '60+'), 0),
             (('三级科员', '0-20'), 0), (('三级科员', '21-30'), 11), (('三级科员', '31-40'), 8)]
data = pd.Series([x[1] for x in data_list], index=[x[0] for x in data_list])
data

(一级科员, 0-20)      0
(一级科员, 21-30)    14
(一级科员, 31-40)    10
(一级科员, 41-50)     5
(一级科员, 51-60)     6
(一级科员, 60+)       0
(三级科员, 0-20)      0
(三级科员, 21-30)    11
(三级科员, 31-40)     8
dtype: int64

In [ ]:
df = data.reset_index() # 要将组合Index的Series转换成DataFrame，需要重置索引
df.columns = ['Combination', 'Value']
df[['Rank', 'Age']] = df['Combination'].apply(pd.Series) # 将组合列拆分成两列
df

,Combination,Value,Rank,Age
0,"(一级科员, 0-20)",0,一级科员,0-20
1,"(一级科员, 21-30)",14,一级科员,21-30
2,"(一级科员, 31-40)",10,一级科员,31-40
3,"(一级科员, 41-50)",5,一级科员,41-50
4,"(一级科员, 51-60)",6,一级科员,51-60
5,"(一级科员, 60+)",0,一级科员,60+
6,"(三级科员, 0-20)",0,三级科员,0-20
7,"(三级科员, 21-30)",11,三级科员,21-30
8,"(三级科员, 31-40)",8,三级科员,31-40


In [ ]:
data.dtypes

dtype('int64')

### DataFrame数据

通过传入一个Numpy数组来创建DataFrame，使用date_range来创建索引数据

注意返回的数据里面有一个freq对象，这个表示时间类数据的频率，D表示日历日频率，常见的有：
- B 工作日频率 
- C 自定义工作日频率
- D 日历日频率
- W 周频率 
- M 月末频率 
- SM 半月结束频率 
- BM 营业月结束频率

In [ ]:
import pandas as pd
import numpy as np

dates = pd.date_range("20130101", periods=6)
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list("ABCD"))
df

,A,B,C,D
2013-01-01,0.737109,-1.512517,-1.889726,-0.903838
2013-01-02,1.012142,2.329710,0.269707,0.569603
2013-01-03,0.071112,-1.318172,-0.582907,0.134474
2013-01-04,0.562635,-0.063299,0.927822,0.071969
2013-01-05,-0.595525,1.406538,0.419552,0.038937
2013-01-06,-0.316271,-0.015060,1.235434,0.041634


In [ ]:
df.sum(axis=1)

2013-01-01   -3.568972
2013-01-02    4.181162
2013-01-03   -1.695493
2013-01-04    1.499127
2013-01-05    1.269501
2013-01-06    0.945737
Freq: D, dtype: float64

In [ ]:
df.gt(0).sum(axis=1)

2013-01-01    1
2013-01-02    4
2013-01-03    2
2013-01-04    3
2013-01-05    3
2013-01-06    2
Freq: D, dtype: int64

查看数据情况(行数,列数)

In [ ]:
df.shape

(6, 4)

In [ ]:
pro = np.product(df.shape)
pro

24

In [ ]:
a = [['a', '1.2', '4.2'], ['b', '70', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df

,one,two,three
0,a,1.2,4.2
1,b,70,0.03
2,x,5,0


通过分隔字符串创建DataFrame

In [ ]:
import pandas as pd
# 使用给定的示例数据
data = '''伍峰	三级科员	2000-10-8	2004-10-13	2006-10-13
何健	四级科员	1978-5-6	2018-6-13	2020-6-13
湛俊	二级科员	1967-9-20	2004-12-11	2008-12-11
柏强	二级科员	1975-7-15	2019-3-1	2022-3-1
孟俊	一级科员	1966-2-14	1992-10-10	1996-10-10'''

df = pd.DataFrame([x.split('\t') for x in data.split('\n')], columns=['name', 'job', 'birth_date', 'entry_date', 'last_promotion_date'])
df.head()

,name,job,birth_date,entry_date,last_promotion_date
0,伍峰,三级科员,2000-10-8,2004-10-13,2006-10-13
1,何健,四级科员,1978-5-6,2018-6-13,2020-6-13
2,湛俊,二级科员,1967-9-20,2004-12-11,2008-12-11
3,柏强,二级科员,1975-7-15,2019-3-1,2022-3-1
4,孟俊,一级科员,1966-2-14,1992-10-10,1996-10-10


通过传入字典对象来创建DataFrame

In [ ]:
people = pd.DataFrame({'Name':['a','b'],'Age':[18, 22]}, index=[0,1])
people

,Name,Age
0,a,18
1,b,22


In [ ]:
loc = pd.DataFrame({'Location':['四川1','重庆']}, index=[0,1])
p2 = people.join(loc)
p2

,Name,Age,Location
0,a,18,四川1
1,b,22,重庆


In [ ]:
df2 = pd.DataFrame({
    "A": 1.0,
    "B": pd.Timestamp("20130102"),
    "C": pd.Series(1, index=list(range(4)), dtype="float32"),
    "D": np.array([3]*4, dtype="int32"),
    "E": pd.Categorical(["test", "train", "test", "train"]),
    "F": "foo",
})
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


In [ ]:
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
dtype: object

In [ ]:
df2.index

Index([0, 1, 2, 3], dtype='int64')

In [ ]:
df.describe()

,name,job,birth_date,entry_date,last_promotion_date
count,5,5,5,5,5
unique,5,4,5,5,5
top,伍峰,二级科员,2000-10-8,2004-10-13,2006-10-13
freq,1,2,1,1,1


转置数据,行列颠倒

In [ ]:
df.T

,0,1,2,3,4
name,伍峰,何健,湛俊,柏强,孟俊
job,三级科员,四级科员,二级科员,二级科员,一级科员
birth_date,2000-10-8,1978-5-6,1967-9-20,1975-7-15,1966-2-14
entry_date,2004-10-13,2018-6-13,2004-12-11,2019-3-1,1992-10-10
last_promotion_date,2006-10-13,2020-6-13,2008-12-11,2022-3-1,1996-10-10


对列进行排序

In [ ]:
df.sort_index(axis=1, ascending=False)

,name,last_promotion_date,job,entry_date,birth_date
0,伍峰,2006-10-13,三级科员,2004-10-13,2000-10-8
1,何健,2020-6-13,四级科员,2018-6-13,1978-5-6
2,湛俊,2008-12-11,二级科员,2004-12-11,1967-9-20
3,柏强,2022-3-1,二级科员,2019-3-1,1975-7-15
4,孟俊,1996-10-10,一级科员,1992-10-10,1966-2-14


对值排序

删除数据, 在删除时可以指定axis, axis=0是按行删除,axis=1是按列删除

In [ ]:
import pandas as pd

# 创建一个包含五行的 DataFrame
data = {'A': [1, 2, 3, 4, 5], 'B': ['a', pd.NA, 'c', 'd', 'e'], 'C': [True,True,True,True,False], 'D': [1.1, 2.3, pd.NA, 4.5, 5.6]}
df = pd.DataFrame(data)
df

,A,B,C,D
0,1,a,True,1.1
1,2,<NA>,True,2.3
2,3,c,True,<NA>
3,4,d,True,4.5
4,5,e,False,5.6


In [ ]:
# 删除第三行
drop_row_df = df.drop(2)
drop_row_df

,A,B,C,D
0,1,a,True,1.1
1,2,<NA>,True,2.3
3,4,d,True,4.5
4,5,e,False,5.6


In [ ]:
# 删除空行
drop_null_row = df.dropna(axis=0)
drop_null_row

,A,B,C,D
0,1,a,True,1.1
3,4,d,True,4.5
4,5,e,False,5.6


In [ ]:
# 只删除指定的，包含空列的行
drop_specified_row = df.dropna(axis=0, subset=['B'])
drop_specified_row

,A,B,C,D
0,1,a,True,1.1
2,3,c,True,<NA>
3,4,d,True,4.5
4,5,e,False,5.6


In [ ]:
# 删除空列
drop_null_col = df.dropna(axis=1)
drop_null_col

,A,C
0,1,True
1,2,True
2,3,True
3,4,True
4,5,False


In [ ]:
# 直接修改原数据
df.dropna(inplace=True)
df

,A,B,C,D
0,1,a,True,1.1
3,4,d,True,4.5
4,5,e,False,5.6


## 转换数据
使用pivot将行转换为列, 使用mellt将数据从列转换为行

pivot()方法可以接受以下参数：

* index：用作新DataFrame的行索引的列名或列名列表。
* columns：用作新DataFrame的列索引的列名或列名列表。
* values：用于填充新DataFrame的列的列名或列名列表。如果未指定，则使用所有其他列。
* aggfunc：用于聚合重复值的函数，例如“sum”或“mean”。

In [ ]:
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emily'],
    'Year': [2016, 2016, 2017, 2017, 2018],
    'Salary': [50000, 55000, 60000, 65000, 70000]
}

df = pd.DataFrame(data)
print('index:', df.index)
print('columns:',df.columns)
df.head()

index: RangeIndex(start=0, stop=5, step=1)
columns: Index(['Name', 'Year', 'Salary'], dtype='object')


,Name,Year,Salary
0,Alice,2016,50000
1,Bob,2016,55000
2,Charlie,2017,60000
3,David,2017,65000
4,Emily,2018,70000


In [ ]:
pivot_df = df.pivot(index='Year', columns='Name', values='Salary')
print('index:', pivot_df.index)
print('columns:',pivot_df.columns)
pivot_df.head()

index: Index([2016, 2017, 2018], dtype='int64', name='Year')
columns: Index(['Alice', 'Bob', 'Charlie', 'David', 'Emily'], dtype='object', name='Name')


Name,Alice,Bob,Charlie,David,Emily
Year,,,,,
2016,50000.0,55000.0,NaN,NaN,NaN
2017,NaN,NaN,60000.0,65000.0,NaN
2018,NaN,NaN,NaN,NaN,70000.0


In [ ]:
reset_df = pivot_df.reset_index()
reset_df.index.name='AutoRangeIndex'
print('index:', reset_df.index)
print('columns:',reset_df.columns)
reset_df.head()

index: RangeIndex(start=0, stop=3, step=1, name='AutoRangeIndex')
columns: Index(['Year', 'Alice', 'Bob', 'Charlie', 'David', 'Emily'], dtype='object', name='Name')


Name,Year,Alice,Bob,Charlie,David,Emily
AutoRangeIndex,,,,,,
0,2016,50000.0,55000.0,NaN,NaN,NaN
1,2017,NaN,NaN,60000.0,65000.0,NaN
2,2018,NaN,NaN,NaN,NaN,70000.0


In [ ]:
melt_df = reset_df.melt(id_vars='Year', var_name='Name1', value_name='Salary')
print('index:', melt_df.index)
print('columns:',melt_df.columns)
melt_df.head()

index: RangeIndex(start=0, stop=15, step=1)
columns: Index(['Year', 'Name1', 'Salary'], dtype='object')


,Year,Name1,Salary
0,2016,Alice,50000.0
1,2017,Alice,NaN
2,2018,Alice,NaN
3,2016,Bob,55000.0
4,2017,Bob,NaN


转换指定的列

In [ ]:
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emily'],
    'Year': [2016, 2016, 2017, 2017, 2018],
    'Salary': [50000, 55000, 60000, 65000, 70000],
    'Bonus': [1000, 2000, 3000, 4000, 5000]
}

df = pd.DataFrame(data)

In [ ]:
melt_df = df.melt(id_vars=['Name', 'Year'], value_vars=['Salary'], var_name='Metric', value_name='Amount')
melt_df.head(50)

,Name,Year,Metric,Amount
0,Alice,2016,Salary,50000
1,Bob,2016,Salary,55000
2,Charlie,2017,Salary,60000
3,David,2017,Salary,65000
4,Emily,2018,Salary,70000


## 数据选择
使用布尔索引和使用 `loc` 方法都可以根据条件获取行，但它们在内存占用方面有一些区别。

1. 布尔索引：使用布尔索引可以直接筛选出满足条件的行，返回一个新的 DataFrame，其中只包含符合条件的行。这种方法不会复制原始数据，因此在内存占用方面相对较小。它是一种有效的方法，特别适用于大型数据集。

```python
# 示例条件：获取 'column_name' 列值大于 10 的行
condition = df['column_name'] > 10
filtered_df = df[condition]
```

2. `loc` 方法：`loc` 方法可以根据标签进行行选择，并返回一个新的 DataFrame。它复制了满足条件的行的数据，因此在内存占用方面可能比布尔索引稍微更大一些。如果数据集非常大，复制所有行的数据可能会占用更多内存。

```python
# 示例条件：获取 'column_name' 列值大于 10 的行
filtered_df = df.loc[df['column_name'] > 10]
```

总体而言，在大多数情况下，布尔索引会占用较少的内存，因为它直接从原始数据中选择和过滤行，而不复制数据。但是，如果你的数据集相对较小，使用 `loc` 方法也是一种方便和直观的方法，不会产生明显的内存问题。

在选择方法时，请考虑数据集的大小、内存限制以及性能需求，并选择最适合你的具体情况的方法。

选择单列，返回一个Series

In [ ]:
import pandas as pd

data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Emily'],
    'Year': [2016, 2016, 2017, 2017, 2018],
    'Salary': [50000, 55000, 60000, 65000, 70000],
    'Bonus': [1000, 2000, 3000, 4000, 5000]
}

df = pd.DataFrame(data)

In [ ]:
df

,Name,Year,Salary,Bonus
0,Alice,2016,50000,1000
1,Bob,2016,55000,2000
2,Charlie,2017,60000,3000
3,David,2017,65000,4000
4,Emily,2018,70000,5000


1. 如果要更改原始数据，请使用单一赋值操作（loc）：

In [ ]:
df.loc[df['Year'] == 2016, 'Salary'] = 200

2. 如果想要一个副本，请确保强制让 Pandas 创建副本：

In [ ]:
df_2016 = df.loc[df['Year'] == 2016].copy()
df_2016.loc[df_2016['Name']=='Alice', 'Bonus'] = 1234
df_2016.head()

,Name,Year,Salary,Bonus
0,Alice,2016,200,1234
1,Bob,2016,200,2000


### 根据正则匹配
使用str.contains包含字符，使用~是不包含

### 返回指定列数据

In [ ]:
df[0:3]

,Name,Year,Salary,Bonus
0,Alice,2016,200,1000
1,Bob,2016,200,2000
2,Charlie,2017,60000,3000


### 根据label获取数据
DataFrame.loc 根据Label获取数据
支持的如下3种输入
* 一个label, e.g. 5
* list label, e.g. ['a','b','c']
* slice object, e.g. 'a':'f'

### iloc 根据位置获取数据
df.iloc根据索引位置和列位置获取数据，注意两个参数都必须是位置，不能一个位置一个label
* 第一个参数是行索引,可以是具体的数字，也可以是区间，比如0:3
* 第二个参数是列索引

In [ ]:
df.iloc[3]

Name      David
Year       2017
Salary    65000
Bonus      4000
Name: 3, dtype: object

In [ ]:
df.iloc[3:5, 0:2]

,Name,Year
3,David,2017
4,Emily,2018


In [ ]:
df.iloc[[1,2,4],[0,2]]

,Name,Salary
1,Bob,200
2,Charlie,60000
4,Emily,70000


In [ ]:
df.iloc[1:3, :]

,Name,Year,Salary,Bonus
1,Bob,2016,200,2000
2,Charlie,2017,60000,3000


In [ ]:
df.iloc[:, 1:3]

,Year,Salary
0,2016,200
1,2016,200
2,2017,60000
3,2017,65000
4,2018,70000


In [ ]:
df.iloc[1,1]

2016

In [ ]:
df.iat[1,1]

2016

## Boolean索引

过滤数据

In [ ]:
df[df["Year"]>2017]

,Name,Year,Salary,Bonus
4,Emily,2018,70000,5000


使用isin()方法过滤数据

In [ ]:
df2 = df.copy()

In [ ]:
s1 = pd.Series([1,2,3,4,5,6], index=pd.date_range("20130102", periods=6))

In [ ]:
s1

2013-01-02    1
2013-01-03    2
2013-01-04    3
2013-01-05    4
2013-01-06    5
2013-01-07    6
Freq: D, dtype: int64

In [ ]:
s2 = pd.Series([11,12,13,14,15,16,17])
s2

0    11
1    12
2    13
3    14
4    15
5    16
6    17
dtype: int64

## 日期处理

In [ ]:
import pandas as pd

# 创建测试数据
df = pd.DataFrame({'date': ['1994.03', '2020.02', '2020.03']})

# 将字符串列转换为datetime列
df['date'] = pd.to_datetime(df['date'], format='%Y.%m')

# 打印转换后的结果
print(df)

        date
0 1994-03-01
1 2020-02-01
2 2020-03-01


设置值

In [ ]:
# 根据label设置
df.at[dates[0],"A"] = 0
df.loc[dates[0]]

date    NaT
A       0.0
Name: 2013-01-01 00:00:00, dtype: object

In [ ]:
# 根据位置设置
df.iat[0,1]=0
df.iloc[0,:]

date    1994-03-01 00:00:00
A                       0.0
Name: 0, dtype: object

In [ ]:
# 设置一个数组
df.loc[:,"D"] = np.array([5]*len(df))

In [ ]:
df.D

0                      5
1                      5
2                      5
2013-01-01 00:00:00    5
Name: D, dtype: int64

## 数据转换
可以使用函数或者Mapping转换数据

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [ ]:
lowercased = data.food.str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [ ]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 替换值

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [ ]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

## 数据缺失
pandas使用np.nan来表示数据缺失

缺失数据处理
对空值的处理有3种办法
1. 去掉包含空值的列
2. 给空值赋值为平均值
3. 赋值的同时，添加一个标明数值缺失的属性

### 给空值赋值为平均值

In [ ]:
from sklearn.impute import SimpleImputer

df_si = pd.DataFrame({'A': [1, np.nan, 2, 3, 4], 'B': [2, 3, 4, 5, 6]})
my_imputer = SimpleImputer()

imputed_df_si = pd.DataFrame(my_imputer.fit_transform(df_si))
imputed_df_si


,0,1
0,1.0,2.0
1,2.5,3.0
2,2.0,4.0
3,3.0,5.0
4,4.0,6.0


In [ ]:
imputed_df_si_2 = pd.DataFrame(my_imputer.transform(df_si))
imputed_df_si_2

,0,1
0,1.0,2.0
1,2.5,3.0
2,2.0,4.0
3,3.0,5.0
4,4.0,6.0


Reindexing 可以根据列进行数据变化，返回一个数据的拷贝

In [ ]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns)+["E"])
df1.loc[dates[0]:dates[1], "E"] = 1
df1

,date,A,D,E
2013-01-01,NaT,0.0,5.0,1.0
2013-01-02,NaT,NaN,NaN,1.0
2013-01-03,NaT,NaN,NaN,NaN
2013-01-04,NaT,NaN,NaN,NaN


In [ ]:
df1.at["20130102","G"]=2

In [ ]:
df1

,date,A,D,E,G
2013-01-01,NaT,0.0,5.0,1.0,NaN
2013-01-02,NaT,NaN,NaN,1.0,2.0
2013-01-03,NaT,NaN,NaN,NaN,NaN
2013-01-04,NaT,NaN,NaN,NaN,NaN


`DataFrame.dropna()`删处任何办好空数据的列，返回删除后的数据，不会改变原数据

In [ ]:
df1.dropna(how="any")

,date,A,D,E,G


In [ ]:
df1

,date,A,D,E,G
2013-01-01,NaT,0.0,5.0,1.0,NaN
2013-01-02,NaT,NaN,NaN,1.0,2.0
2013-01-03,NaT,NaN,NaN,NaN,NaN
2013-01-04,NaT,NaN,NaN,NaN,NaN


`DataFrame.fillna`填充缺失的数据

In [ ]:
df1.fillna(value=5)

,date,A,D,E,G
2013-01-01,5,0.0,5.0,1.0,5.0
2013-01-02,5,5.0,5.0,1.0,2.0
2013-01-03,5,5.0,5.0,5.0,5.0
2013-01-04,5,5.0,5.0,5.0,5.0


`isna()`获取nan的boolean数据

In [ ]:
pd.isna(df1)

,date,A,D,E,G
2013-01-01,True,False,False,False,True
2013-01-02,True,True,True,False,False
2013-01-03,True,True,True,True,True
2013-01-04,True,True,True,True,True


In [ ]:
s = pd.Series(["8", 6, "7.5", 3, "0.9"]) # mixed string and numeric values
s.dtypes

dtype('O')

In [ ]:
pd.to_numeric(s) # convert everything to float values
s.dtypes

dtype('O')

## 排序

In [ ]:
df = pd.DataFrame({
    'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
    'col4': ['a', 'B', 'c', 'D', 'e', 'F']
})
df.sort_values(by=['col1','col2'], ascending=True)

,col1,col2,col3,col4
1,A,1,1,B
0,A,2,0,a
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


要恢复排序可以使用reset_index

In [ ]:
df.reset_index(drop=True)

,col1,col2,col3,col4
0,A,2,0,a
1,A,1,1,B
2,B,9,9,c
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F


## 索引

In [ ]:
df = pd.DataFrame([('bird', 389.0),
                   ('bird', 24.0),
                   ('mammal', 80.5),
                   ('mammal', np.nan)],
                  index=['falcon', 'parrot', 'lion', 'monkey'],
                  columns=('class', 'max_speed'))
df

,class,max_speed
falcon,bird,389.0
parrot,bird,24.0
lion,mammal,80.5
monkey,mammal,NaN


当我们执行reset_index后，旧的index会作用一列添加到数据中

In [ ]:
df.reset_index()

,index,class,max_speed
0,falcon,bird,389.0
1,parrot,bird,24.0
2,lion,mammal,80.5
3,monkey,mammal,NaN


我们可以使用drop参数移除索引

In [ ]:
df.reset_index(drop=True)

,class,max_speed
0,bird,389.0
1,bird,24.0
2,mammal,80.5
3,mammal,NaN


### Apply
`DataFrame.apply()`应用一个用户自定义的函数到数据上

In [ ]:
df

,class,max_speed
falcon,bird,389.0
parrot,bird,24.0
lion,mammal,80.5
monkey,mammal,NaN


In [ ]:
df.apply(np.cumsum)

,class,max_speed
falcon,bird,389.0
parrot,birdbird,413.0
lion,birdbirdmammal,493.5
monkey,birdbirdmammalmammal,NaN


## Pandas函数

DataFrame.corr(method='pearson', min_periods=1) 计算相关系数

参数说明：

method：可选值为{‘pearson’, ‘kendall’, ‘spearman’}

- pearson：Pearson相关系数来衡量两个数据集合是否在一条线上面，即针对线性数据的相关系数计算，针对非线性数据便会有误差。
- kendall：用于反映分类变量相关性的指标，即针对无序序列的相关系数，非正太分布的数据
- spearman：非线性的，非正太分布的数据的相关系数

min_periods：样本最少的数据量

返回值：各类型之间的相关系数DataFrame表格。

两组数据间的相关性计算可以分为如下3种情况：

1. 数值数据与分类数据
2. 数值数据与数值数据
3. 分类数据与分类数据



### 数值与数值的相关性

In [ ]:
import pandas as pd
 
data = pd.DataFrame({'化妆品费': [30, 50, 120, 20, 70, 150, 50, 60, 80, 100],
                     '置装费': [70, 80, 250, 50, 120, 300, 100, 150, 20, 180]})
print(data.corr()) # 计算所有的变量的两两相关性
print(data['化妆品费'].corr(data['置装费'])) # 只计算选择的两个变量的相关性

          化妆品费       置装费
化妆品费  1.000000  0.850918
置装费   0.850918  1.000000
0.8509180035311159


### 数值与分类的相关性

In [ ]:
# 情况1：分类标签为数字
data = pd.DataFrame({'id': [3, 2, 1, 1, 2, 3, 2, 3, 1, 1, 2, 3, 1, 2, 1],
                     'age': [27, 33, 16, 29, 32, 23, 25, 28, 22, 18, 26, 26, 15, 29, 26]})
print('pearson:', data['id'].corr(data['age']))
print('spearman', data['id'].corr(data['age'], method='spearman'))
 
# 情况2：分类标签为字符串
data1 = pd.DataFrame({'id': ['c', 'b', 'a', 'a', 'b', 'c', 'b', 'c', 'a', 'a', 'b', 'c', 'a', 'b', 'a'],
                     'age': [27, 33, 16, 29, 32, 23, 25, 28, 22, 18, 26, 26, 15, 29, 26]})
print('spearman', data1['id'].corr(data1['age'], method='spearman'))
 
# 输出
# pearson: 0.4465155114816965
# spearman 0.4016086046008866
# spearman 0.4016086046008866

pearson: 0.4465155114816965
spearman 0.4016086046008866
spearman 0.4016086046008866
